# Consumo api con llm como extractor de información

## Imports

In [1]:
import os
import requests
import vertexai
from dotenv import load_dotenv
from langchain_google_vertexai import ChatVertexAI
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

## LLM

In [2]:

vertexai.init(
    project="gen-lang-client-0730997933",
    location="us-central1"
)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gen-lang-client-0730997933-9e99e8511560.json'

load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm_gemini_pro = ChatVertexAI(model="gemini-1.5-pro-002")

## Funcioon para busqueda de pokemon por nombre consumiendo api

In [132]:
def get_pokemon_info(pokename):
    url=f'https://pokeapi.co/api/v2/pokemon/{pokename}'
    headers = {
        "accept": "application/json"
    }
    response =  requests.get(url, headers=headers)
    return response

In [133]:
pokename = 'ditto'
info_pokemon = get_pokemon_info(pokename)

In [134]:
info_json_abilities = info_pokemon.json().get('abilities')

In [135]:
info_json_abilities

[{'ability': {'name': 'limber', 'url': 'https://pokeapi.co/api/v2/ability/7/'},
  'is_hidden': False,
  'slot': 1},
 {'ability': {'name': 'imposter',
   'url': 'https://pokeapi.co/api/v2/ability/150/'},
  'is_hidden': True,
  'slot': 3}]

In [137]:
import json

info_json_abilities = json.dumps(info_json_abilities)

## Prompt

In [171]:
template = "Entragame informacion sobre {pokename}, quiero que me estructures la informacion y me entregues un resumen en de este y ademas un listados de las habilidades que puede aprender. La información es la siguiente: {info_json_abilities}"

In [172]:
prompt_template = PromptTemplate(
    input_variables = ['pokename', 'info_json_abilities'], template = template
)

In [ ]:
print(prompt_template.format(pokename= 'ditto', info_json_abilities= info_json_abilities))

Entragame informacion sobre ditto, quiero que me estructures la informacion y me entregues un resumen en de este y ademas un listados de las habilidades que puede aprender. La información es la siguiente: "[{\"ability\": {\"name\": \"limber\", \"url\": \"https://pokeapi.co/api/v2/ability/7/\"}, \"is_hidden\": false, \"slot\": 1}, {\"ability\": {\"name\": \"imposter\", \"url\": \"https://pokeapi.co/api/v2/ability/150/\"}, \"is_hidden\": true, \"slot\": 3}]"


## Chain

In [173]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt_template | llm_gemini_pro | StrOutputParser()

In [181]:
result = chain.invoke(input= {'pokename': 'ditto', 'info_json_abilities': info_json_abilities})

In [182]:
result

'## Ditto: El Pokémon Transformación\n\n**Resumen:**\n\nDitto es un Pokémon conocido por su asombrosa capacidad de transformarse en otros Pokémon.  Si bien su forma base es una simple mancha rosa amorfa,  puede copiar la apariencia y las habilidades de cualquier Pokémon que vea.  Su habilidad principal, "Flexibilidad", lo protege de la parálisis.  Sin embargo, su habilidad oculta, "Impostor", es aún más impresionante, permitiéndole transformarse automáticamente en el Pokémon oponente al entrar en combate.\n\n**Estructura de la información:**\n\nLa información proporcionada describe las habilidades de Ditto en un formato JSON.  Cada objeto dentro del array representa una habilidad:\n\n* **`ability`**: Contiene el nombre y la URL de la API para más información sobre la habilidad.\n* **`name`**: Nombre de la habilidad.\n* **`url`**: URL de la API con información detallada de la habilidad.\n* **`is_hidden`**: Indica si la habilidad es oculta o no (verdadero o falso).\n* **`slot`**: Represe